In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import FloatVGG16
from SampleNN_dev import *
from tqdm import tqdm_notebook
%matplotlib inline
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import torchvision
import SampleVGG16_dev

In [2]:
import torchvision.transforms as transforms
traindir = '/home/data/yanzy/train'
valdir = '/home/data/yanzy/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers = 32, pin_memory=True)

train_dataset =  torchvision.datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,
                                         shuffle=True, num_workers = 16, pin_memory=False)

In [3]:
float_net = torchvision.models.vgg16(pretrained=True).to(device)
def val(Sample = 1, m = 6):
    sample_net = SampleVGG16_dev.SampleVGG16(m = m).to(device)
    lol = sample_net.state_dict()
    a = lol.keys()
    gt  = float_net.state_dict()
    b = gt.keys()
    for i in  range(32):
        lol[a[i]] = gt[b[i]]
    sample_net.load_state_dict(lol)
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net,N = 16, m = m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            if count % Sample == 0:
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = oSample()(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total
            count += 1

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    return 100.0 * correct / total

In [5]:
for i in range (6,16):
    val(Sample = 1, m = i)

When m = 6, Accuracy of the network on the 50000 test images: 60.636 %


When m = 7, Accuracy of the network on the 50000 test images: 62.816 %


When m = 8, Accuracy of the network on the 50000 test images: 63.358 %


When m = 9, Accuracy of the network on the 50000 test images: 63.120 %


When m = 10, Accuracy of the network on the 50000 test images: 56.228 %


When m = 11, Accuracy of the network on the 50000 test images: 29.654 %


When m = 12, Accuracy of the network on the 50000 test images: 1.856 %


When m = 13, Accuracy of the network on the 50000 test images: 0.296 %


When m = 14, Accuracy of the network on the 50000 test images: 0.138 %


When m = 15, Accuracy of the network on the 50000 test images: 0.102 %
